# ELEC 598 : Deep Learning Applications 
# Project 1 - Predicting Housing Prices 
### By: Matthew Stein and Ethan Tsao 
### February 10, 2020 

In [69]:
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data Preprocessing 

Data is imported via pandas, and processed via numpy. Working with Keras, all data is processed as arrays in numpy. To simplify model learning, feature-wise normalization is performed. Note that feature-wise normalization is calc'ed using the mean and std of the training set. 

The test set was chosen to be 15% of the overall data set. 

In [73]:
data = pd.read_excel('data/resd_data.xlsx')
length = data.shape[0]

# Remove all rows with null points in dataset 
updated_data = data.dropna(how='any',axis=0) 


# Remove all rows with "0" for the Postal Code, "0" for Year Built, "0" for Lot Square Footage, and "0" for List Price
updated_data = updated_data[updated_data.field_PostalCode != 0 ]
updated_data = updated_data[updated_data.YearBuilt != 0]
updated_data = updated_data[updated_data.LotSquareFootage != 0]
updated_data = updated_data[updated_data.ListPrice != 0]

updated_data['ListPrice'] = updated_data['ListPrice']/1000 

# Update Index column 

print(updated_data) 

      SquareFootageStructure  LotSquareFootage  YearBuilt  Bedrooms  \
0                     1549.0            5825.0     1974.0       3.0   
1                     1196.0            7900.0     1981.0       3.0   
3                     3884.0           16013.0     1978.0       3.0   
6                     1164.0            6611.0     1961.0       3.0   
12                    2766.0            7331.0     1986.0       4.0   
...                      ...               ...        ...       ...   
9989                  2300.0            6547.0     1924.0       4.0   
9990                  1545.0            2178.0     1967.0       2.0   
9991                  1404.0            7841.0     1964.0       2.0   
9993                   900.0            6000.0     1924.0       2.0   
9999                  2865.0            7841.0     2005.0       4.0   

      BathsTotal  field_StoriesTotal  field_PostalCode  ListPrice  
0            2.0                 1.0           92624.0      3.300  
1          

In [74]:
train_data = updated_data.loc[:(6370*.85),:'field_PostalCode'].values.tolist()
train_data = np.asarray(train_data) 
train_targets = updated_data.loc[:(6370*.85),'ListPrice'].values.tolist()
train_targets = np.asarray(train_targets) 

test_data = updated_data.loc[(6370*.85 +1 ):,:'field_PostalCode'].values.tolist()
test_data = np.asarray(test_data) 
test_targets = updated_data.loc[(6370*.85 + 1):,'ListPrice'].values.tolist()
test_targets = np.asarray(test_targets)

In [75]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

## Model Selection 

As is standard for neural networks - and regression models, in particular - a Sequential model was chosen. The model has 3 hidden layers, with 64 

In [86]:
keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)


def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

NameError: name 'keras' is not defined

## Training and Validation 

In [85]:
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_mae_histories = []


for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    print(train_data[:i * num_val_samples].shape)
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=32, verbose=1)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1) 
    #mae_history = history.history['val_mean_absolute_error']
    #all_mae_histories.append(mae_history)

processing fold # 0
(0, 7)
Train on 1723 samples, validate on 1722 samples
Epoch 1/100
1723/1723 [==============================] - 1s 607us/step - loss: 3585124.6162 - mae: 865.9322 - val_loss: 2695631.2949 - val_mae: 805.0233
Epoch 2/100
1723/1723 [==============================] - 0s 129us/step - loss: 2935374.4048 - mae: 694.8428 - val_loss: 1857640.8595 - val_mae: 600.0143
Epoch 3/100
1723/1723 [==============================] - 0s 128us/step - loss: 2420348.7830 - mae: 604.6332 - val_loss: 1615223.0575 - val_mae: 590.6235
Epoch 4/100
1723/1723 [==============================] - 0s 146us/step - loss: 2258982.6111 - mae: 605.2721 - val_loss: 1507130.4241 - val_mae: 588.4382
Epoch 5/100
1723/1723 [==============================] - 0s 142us/step - loss: 2184732.2699 - mae: 597.2110 - val_loss: 1469044.8773 - val_mae: 577.4031
Epoch 6/100
1723/1723 [==============================] - 0s 134us/step - loss: 2155337.9615 - mae: 609.1622 - val_loss: 1446212.0136 - val_mae: 558.0206
Epoch 7

Epoch 54/100
1723/1723 [==============================] - 0s 128us/step - loss: 1657561.0179 - mae: 534.6322 - val_loss: 1258420.3684 - val_mae: 535.9515
Epoch 55/100
1723/1723 [==============================] - 0s 128us/step - loss: 1649873.4486 - mae: 531.9629 - val_loss: 1262156.2847 - val_mae: 537.2930
Epoch 56/100
1723/1723 [==============================] - 0s 137us/step - loss: 1649215.4061 - mae: 531.8745 - val_loss: 1257733.7616 - val_mae: 533.5933
Epoch 57/100
1723/1723 [==============================] - 0s 130us/step - loss: 1644589.0511 - mae: 527.9104 - val_loss: 1254568.1806 - val_mae: 531.1855
Epoch 58/100
1723/1723 [==============================] - 0s 138us/step - loss: 1637358.2793 - mae: 523.1564 - val_loss: 1259319.7170 - val_mae: 537.9476
Epoch 59/100
1723/1723 [==============================] - 0s 139us/step - loss: 1624540.4121 - mae: 529.4323 - val_loss: 1262593.6825 - val_mae: 532.9222
Epoch 60/100
1723/1723 [==============================] - 0s 128us/step - lo

1723/1723 [==============================] - 0s 143us/step - loss: 1437691.2843 - mae: 563.5240 - val_loss: 2179280.8540 - val_mae: 586.5085
Epoch 7/100
1723/1723 [==============================] - 0s 134us/step - loss: 1416369.3718 - mae: 563.5542 - val_loss: 2158552.6356 - val_mae: 588.5964
Epoch 8/100
1723/1723 [==============================] - 0s 139us/step - loss: 1396988.8692 - mae: 560.7629 - val_loss: 2145032.0407 - val_mae: 587.6395
Epoch 9/100
1723/1723 [==============================] - 0s 139us/step - loss: 1379156.3600 - mae: 555.0150 - val_loss: 2135569.0640 - val_mae: 594.5838
Epoch 10/100
1723/1723 [==============================] - 0s 139us/step - loss: 1374330.7307 - mae: 554.9598 - val_loss: 2123071.1070 - val_mae: 582.2834
Epoch 11/100
1723/1723 [==============================] - 0s 133us/step - loss: 1360351.3931 - mae: 549.1420 - val_loss: 2135744.1094 - val_mae: 607.3796
Epoch 12/100
1723/1723 [==============================] - 0s 137us/step - loss: 1341570.1723

KeyboardInterrupt: 

## Results Discussion 